<a href="https://colab.research.google.com/github/anamika2000/Time-Series-Forecasting-Using-LSTM/blob/main/Time_Series_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Time Series Forecasting 


Time series are widely used for non-stationary data, like economic, weather, stock price, and retail sales in this post. I am going to predict temperature with time series forecasting using RNN method

Time Series Forecasting using CNN-LSTM


In [ ]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense,RepeatVector
from keras.layers import Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [ ]:
data=pd.read_csv('testset.csv.zip')

In [ ]:
data.columns

Index(['datetime_utc', ' _conds', ' _dewptm', ' _fog', ' _hail',
       ' _heatindexm', ' _hum', ' _precipm', ' _pressurem', ' _rain', ' _snow',
       ' _tempm', ' _thunder', ' _tornado', ' _vism', ' _wdird', ' _wdire',
       ' _wgustm', ' _windchillm', ' _wspdm'],
      dtype='object')

In [ ]:
data.head()

,datetime_utc,_conds,_dewptm,_fog,_hail,_heatindexm,_hum,_precipm,_pressurem,_rain,_snow,_tempm,_thunder,_tornado,_vism,_wdird,_wdire,_wgustm,_windchillm,_wspdm
0,19961101-11:00,Smoke,9.0,0,0,NaN,27.0,NaN,1010.0,0,0,30.0,0,0,5.0,280.0,West,NaN,NaN,7.4
1,19961101-12:00,Smoke,10.0,0,0,NaN,32.0,NaN,-9999.0,0,0,28.0,0,0,NaN,0.0,North,NaN,NaN,NaN
2,19961101-13:00,Smoke,11.0,0,0,NaN,44.0,NaN,-9999.0,0,0,24.0,0,0,NaN,0.0,North,NaN,NaN,NaN
3,19961101-14:00,Smoke,10.0,0,0,NaN,41.0,NaN,1010.0,0,0,24.0,0,0,2.0,0.0,North,NaN,NaN,NaN
4,19961101-16:00,Smoke,11.0,0,0,NaN,47.0,NaN,1011.0,0,0,23.0,0,0,1.2,0.0,North,NaN,NaN,0.0


In [ ]:
data['datetime_utc'] = pd.to_datetime(data['datetime_utc'])
data.set_index('datetime_utc', inplace= True)
data =data.resample('D').mean()


In [ ]:
data = data[[' _tempm']]
data[' _tempm'].fillna(data[' _tempm'].mean(),inplace=True)
#data[' _hum'].fillna(data[' _hum'].mean(),inplace=True)



In [ ]:
data.head()

,_tempm
datetime_utc,
1996-11-01,22.333333
1996-11-02,22.916667
1996-11-03,21.791667
1996-11-04,22.722222
1996-11-05,27.800000


#Time Series Forecast using LSTM

Long Short Term Memory networks – usually just called “LSTMs” – are a special kind of RNN, capable of learning long-term dependencies.

LSTMs are explicitly designed to avoid the long-term dependency problem. Remembering information for long periods of time is practically their default behavior, not something they struggle to learn.

In [ ]:
data=data.astype('float32')

In [ ]:
scaler= MinMaxScaler(feature_range=(-1,1))
sc= scaler.fit_transform(data)
sc
#len(sc)

array([[-0.07224715],
       [-0.03911316],
       [-0.10301447],
       ...,
       [ 0.5279522 ],
       [ 0.5265322 ],
       [ 0.47683108]], dtype=float32)

In [ ]:
timestep=30
x=[]
y=[]
for i in range(len(sc)-(timestep)):
  x.append(sc[i:i+timestep])
  y.append(sc[i+timestep])

In [ ]:
x=np.asanyarray(x)
y=np.asanyarray(y)
print(x.shape)
y.shape

(7450, 30, 1)


(7450, 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.02)
print(X_train.shape)
print(X_test.shape)
#print(X_train[0])
#print(X_test[1])
y_test[0]



(7301, 30, 1)
(149, 30, 1)


array([0.618834], dtype=float32)

#CNN - LSTM Model

In [ ]:
model = Sequential()
model.add(Conv1D(filters=256,kernel_size=2,activation='relu',input_shape=(30,1)))
model.add(Conv1D(filters=128,kernel_size=2,activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(RepeatVector(30))
model.add(LSTM(100,activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(1))


In [ ]:
model.compile(loss='mse',optimizer='adam')

In [ ]:
model.fit(X_train,y_train,epochs=1,verbose=0)


In [ ]:
accuracy=model.evaluate(X_test,y_test,verbose=1)

5/5 [==============================] - 0s 34ms/step - loss: 0.0135 - accuracy: 0.0000e+00


In [ ]:

pred=model.predict(X_test[3:4])
pred=scaler.inverse_transform(pred)
print(pred)
#print(y_test[0])


#y_test=np.asanyarray(y_test) 
#y_test=y_test.reshape(-1,1) 
#y_test[0]
y_test = scaler.inverse_transform(y_test)
y_test[3]


#mean_squared_error(y_test,pred)


[[17.962585]]


array([53987920.], dtype=float32)

Predicting Temperature in test dataset

In [ ]:
data1=pd.read_csv('testset3.csv')
data1.set_index('date', inplace= True)
data1 = data1[['temp']]


In [ ]:
data1.head()

,temp
date,
25-09-2020,18.111
26-09-2020,22.278
27-09-2020,20.000
28-09-2020,15.778
29-09-2020,13.442


In [ ]:
data1=data1.astype('float32')
scaler= MinMaxScaler(feature_range=(-1,1))
scl= scaler.fit_transform(data1)

In [ ]:
steps=30
xi=[]
yi=[]
for i in range(len(scl)-(steps)):
  xi.append(scl[i:i+steps])
  yi.append(scl[i+steps])
xi=np.asanyarray(xi)
yi=np.asanyarray(yi)
#xi.shape
yi



array([[-0.41496962],
       [-0.7791628 ],
       [-1.        ],
       [-0.659771  ],
       [-0.516415  ],
       [-0.8627693 ]], dtype=float32)

In [ ]:
def end_input(inp,new_input):   
    for i in range(steps-1):
        inp[:,i,:] = inp[:,i+1,:]
      
    inp[:,steps-1,:] = new_input
    return inp
future=1
predicted = []
inp = xi[0:1,:,:]
for i in range(future):
    output = new_model.predict(inp, batch_size=1) 
    op=scaler.inverse_transform(output)
    print(op)
    #predicted.append(op)
    inp = end_input(inp,output[0,0]) 


[[9.532641]]
